In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
df = pd.read_excel('ResMetBE20.xlsx', sheet_name=[0, 1])[1].sort_values(by='nommetier')

In [ ]:
# Carte métier

def creerSeriesMetier(metier):
    data_par_metier = df[df['nommetier']==metier]
    new_dict = {}
    for d in data_par_metier['Dept'].unique():
        s = sum([int(k) for k in data_par_metier[data_par_metier['Dept']==d]['met'] if k!='*'])
        new_dict[d]=s
    return pd.Series(data=new_dict)

def creerCarteMetier(metier):
    
    projets_recrutements_par_metier = creerSeriesMetier(metier)

    GEOJSON = "departements.geojson"

    
    m = folium.Map(location=[46.70, 1.6], tiles="cartodbpositron", width=600, height=600, zoom_start=6, zoom_control=False,
               scrollWheelZoom=True,
               dragging=True)
    
    fc = folium.Choropleth(
        geo_data=GEOJSON,
        name='choropleth',
        data=projets_recrutements_par_metier,
        key_on='feature.properties.code',
        fill_color='YlGn',
        fill_opacity=1.0,
        line_opacity=0.1,
        nan_fill_color = 'red',
        nan_fill_opacity=1.0,
        legend_name='Projets de recrutement '+ metier.replace(r"'", r"\'"),
        
    ).add_to(m)
    fc.geojson.add_child(
        folium.features.GeoJsonTooltip(aliases=['Département :'],fields=['nom']))
    
    display(m)

In [ ]:
# Top 30 département

def creerSeriesDept(dept):
    df_dept = df[df['NomDept']==dept.upper()]
    new_dict = {}
    for m in df_dept['nommetier'].unique():
        s = sum([int(k) for k in df_dept[df_dept['nommetier']==m]['met'] if k!='*'])
        new_dict[m]=s
    return pd.Series(data=new_dict).sort_values(ascending=False)

def afficherBarPlotTotal(dept):
    series=creerSeriesDept(dept)
    sns.set_style("darkgrid", {"axes.facecolor": ".9"})
    sns.set_context("talk", font_scale=1.4)
    plt.figure(figsize=(15, 50))
    g = sns.barplot(x=series, y=series.index, palette='Blues_d')
    g.set_xticks(range(0, series[0], int(series[0]/20)))
    g.set_xticklabels(labels = range(0, series[0], int(series[0]/20)), rotation=90)
    return g
    
def afficherBarPlotTop30(dept):
    series=creerSeriesDept(dept)
    sns.set_style("darkgrid", {"axes.facecolor": ".9"})
    sns.set_context("talk", font_scale=1.4)
    plt.figure(figsize=(15, 20))
    g = sns.barplot(x=series[:30], y=series[:30].index, palette='Greens_r')
    g.set_xticks(range(0, series[0], int(series[0]/20)))
    g.set_xticklabels(labels = range(0, series[0], int(series[0]/20)), rotation=90)
    g.tick_params(labelsize=22)
    return g

In [ ]:
liste_metiers = sorted(list(df['nommetier'].unique()))
departements = list(df['NomDept'].unique())
liste_departements = [i.capitalize() for i in departements if type(i) is not float]
liste_departements = sorted(liste_departements)


names = ['Top 30 métiers', 'Carte métier']
tab = widgets.Tab(layout=widgets.Layout(width='90%', height='70%'))
[tab.set_title(i, title) for i, title in enumerate(names)]



departement = widgets.Dropdown(
        options=liste_departements,
        value=liste_departements[0],
        description = " ",
        layout=widgets.Layout(width='50%', height='30px', margin='40px')
    )


metier = widgets.Dropdown(
        options=liste_metiers,
        value=liste_metiers[0],
        description=" ",
        layout=widgets.Layout(width='50%', height='30px', margin='40px')
    )


metiers_top30 = interactive(afficherBarPlotTop30, dept=departement)
label_metiers_top30 = widgets.HTML(value = f"<div style='font-size:20px;'>Visualisez les 30 métiers les plus recherchés dans le département choisi dans la liste ci-dessous :</div>")
box_metiers_top30 = widgets.HBox([widgets.Label(value="", layout=widgets.Layout(width='200px')), metiers_top30])

carte_metier = interactive(creerCarteMetier, metier=metier)
label_carte_metier = widgets.HTML(value = f"<div style='font-size:20px;'>Visualisez la carte des projets de recrutement pour le métier choisi dans la liste ci-dessous :</div>")
box_carte_metier = widgets.HBox([widgets.Label(value="", layout=widgets.Layout(width='200px')), carte_metier])

#label_saut_ligne = widgets.Label("   ")

label_titre = widgets.HTML(value = f"<div style='text-align:center;font-size:40px;padding-bottom:40px;'>Outils de visualisation des projets recrutements 2020</div>")

display(label_titre)

tab.children = [widgets.VBox([label_metiers_top30, box_metiers_top30]), widgets.VBox([label_carte_metier, box_carte_metier])]

display(tab)